In [2]:
import pandas as pd
import os

In [6]:
covid_path = "Dataset/covid/annotations"
politics_path = "Dataset/politics/annotations"

covid_anno = pd.read_json(f"../{covid_path}/train.jsonl", lines = True).reset_index(drop=True)
politics_anno = pd.read_json(f"../{politics_path}/train.jsonl", lines = True).reset_index(drop=True)

covid_entities = covid_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)
politics_entities = politics_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)

In [7]:
entity_dict = {}
def count_entity(x):
    if x is not None and len(x) > 0:
        for item in x:
            key = item.lower()
            if key in entity_dict:
                entity_dict[key] = entity_dict[key] + 1
            else:
                entity_dict[key] = 1

_=covid_entities.apply(lambda x: count_entity(x))
_=politics_entities.apply(lambda x: count_entity(x))

#### Consider entities that are present more than 5 times

In [6]:
filtered_entities = {}
for key, value in entity_dict.items():
    if value >= 5:
        filtered_entities[key] = value

In [12]:
covid_anno

In [19]:
# Iterate annotations and modify the dataset
def modify_annotations(data):
    def update_entites(sample_entities):
        _updated_entities = []
        for _item in sample_entities:
            key = _item.lower()
            if key in filtered_entities:
                _updated_entities.append(key)
            else:
                _updated_entities.append("background")
        return _updated_entities

    data.hero = data.hero.apply(lambda x: update_entites(x))
    data.villain = data.villain.apply(lambda x: update_entites(x))
    data.victim = data.victim.apply(lambda x: update_entites(x))
    data.other = data.other.apply(lambda x: update_entites(x))

    return data

covid_ds = modify_annotations(covid_anno.copy())
politics_ds = modify_annotations(politics_anno.copy())

In [21]:
complete_ds = pd.concat([covid_ds, politics_ds], axis = 0,ignore_index=True)

In [22]:
complete_ds.shape

(5552, 6)

In [23]:
import torch
from transformers import BertTokenizer, VisualBertModel

model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

inputs = tokenizer("What is the man eating?", return_tensors="pt")
# this is a custom function that returns the visual embeddings given the image path
visual_embeds = get_visual_embeddings(image_path)

visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long)
visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float)
inputs.update(
    {
        "visual_embeds": visual_embeds,
        "visual_token_type_ids": visual_token_type_ids,
        "visual_attention_mask": visual_attention_mask,
    }
)
outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state

Downloading: 100%|██████████| 631/631 [00:00<00:00, 214kB/s]
Downloading: 100%|██████████| 428M/428M [04:31<00:00, 1.65MB/s]    
Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a 

NameError: name 'get_visual_embeddings' is not defined